In [1]:
import os
import numpy as np
import pandas as pd
import category_encoders as ce
import math
import copy

In [2]:
cd ~/Ames/Ames

C:\Users\scottwi\Ames\Ames


In [3]:
root_directory = os.getcwd()
test_data_path = os.path.join(root_directory, os.path.join('data','raw', 'test1.csv'))
train_data_path = os.path.join(root_directory, os.path.join('data', 'raw', 'train1.csv'))

In [4]:
test_dataframe = pd.read_csv(test_data_path)
train_dataframe = pd.read_csv(train_data_path)


In [5]:
test_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 81 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
MSZoning         1455 non-null object
LotFrontage      1232 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null object
Alley            107 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1457 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1458 non-

#### omit:

street

lot shape

utilities

LandSlope (same as contour?)

roofStyle

Exterior 2nd

BsmtExposure

BsmtFinType2

Heating - a little sparse.

Functional? 

GarageQual - too sparse no pattern.

GarageCond - too sparse.

Fence?

MiscFeature

PoolQC - only 7 not null

### Main function clean_dataset

In [6]:
def clean_dataset(df):
    df = rename_invalid_labels(df)
    df = omit(df)
    df = clean_continuous(df)
    df = clean_catagorical(df)
    df = binary_neighborhood(df)
    df = neighborhood2(df)
    return df

In [7]:
def rename_invalid_labels(df):
    df = df.rename(columns={"1stFlrSF": "FirstFlrSF", "2ndFlrSF": "SecondFlrSF", "3SsnPorch": "ThreeSsnPorch"})
    return df

In [8]:
def omit(df):
    df = df.drop(columns=['Street',
             'LotShape',
             'Utilities',
             'LandSlope',
             'RoofStyle',
             'Exterior2nd',
             'BsmtExposure',
             'BsmtFinType2',
             'Heating',
             'Functional',
             'GarageQual',
             'GarageCond',
             'Fence',
             'MiscFeature',
             'ThreeSsnPorch',
             'PoolQC',
              ], axis=1)
    return df

In [9]:
def relabel_continuous(df):
    df['SumSqft'] = df['FirstFlrSF'] + df['SecondFlrSF']
    df.drop(['FirstFlrSF', 'SecondFlrSF'], axis=1)
    return df

In [10]:
def clean_continuous(df):
    df = relabel_continuous(df)
    df = find_missing_lot_frontage(df)
    df = lot_frontage_replace(df)
    return df

In [11]:
def clean_catagorical(df):
    df = one_hot(df)
    df = ordinal_to_cat_main(df)
    df = cat_to_ordinal(df)
    return df

In [12]:
def find_missing_lot_frontage(df):
    #find the average lot front by neighborhood
    #take the mean lot frontage value for each neighborhood. 
    mean_df = df.groupby(by = ['Neighborhood'], axis=0,).mean()['LotFrontage']
    mean_dict = mean_df.to_dict()
    df.LotFrontage = df.LotFrontage.fillna(df.Neighborhood.map(mean_dict))
    return df

#### keep catagorical and one hot:
MSZoning

alley

land contour

lotConfig

MasVnrType

Foundation

In [13]:
def one_hot(df):
    ce_one_hot_encoder = ce.OneHotEncoder(cols=['MSZoning', 'Alley', 'LandContour', 'LotConfig', 'MasVnrType', 'Foundation'])
    y = df['SalePrice']
    X = df.drop('SalePrice', axis=1)
    df = ce_one_hot_encoder.fit_transform(X, y)
    df = pd.concat([df, y], axis=1)
    return df
    

### Convert neighborhood to both ordinal and binary

In [14]:
def binary_neighborhood(df):
    #break down neighborhoods into a binary encoding.
    df_copy = df.copy()
    y = df['SalePrice']
    X = df_copy.drop('SalePrice', axis=1)
    be = ce.BinaryEncoder(cols=['Neighborhood'])
    be.fit(X, y)
    new_X = be.transform(X)
    return pd.concat([new_X, y], axis = 1)

In [15]:
#binary(train_dataframe)

In [16]:
#find the dictionary with the key as the neighborhood and the value as the average sale price.
'''
neighborhood_list = []
for i in train_dataframe['Neighborhood'].unique():
    neighborhood_list.append(i)
    
neighborhood_list

df = pd.DataFrame(data = neighborhood_list, columns=["n_name"])

zero_array = []
for i in range(0,25):
    zero_array.append(0)
zero_df = pd.DataFrame(data= zero_array, columns=['n_avg_price'])
df = pd.concat([df, zero_df], axis = 1)

for row in df.iterrows():
    pass
    #print(row[1][0])

neighborhood_averages_series = train_dataframe.groupby('Neighborhood')['SalePrice'].mean()
neighborhood_averages_df = neighborhood_averages_series.to_frame()
neighborhood_dict = neighborhood_averages_df.to_dict()
neighborhood_dict
'''
'''
#Find the dictionary with key as the neighborhood and value as the lot frontage.
neighborhood_list = []
for i in train_dataframe['Neighborhood'].unique():
    neighborhood_list.append(i)

#create a new dataframe with neighborhoods
df = pd.DataFrame(data = neighborhood_list, columns = ["n_name"])

#create a new column for lot frontage
zero_array = []
for i in range(0, len(neighborhood_list)):
    zero_array.append(0)
#turn list of zeros into data frame
zero_df = pd.DataFrame(data = zero_array, columns=["n_avg_frontage"])
#join neighborhoods and zeros
df = pd.concat([df, zero_df], axis = 1)

neighborhood_average_lf_df = train_dataframe.groupby('Neighborhood')['LotFrontage'].mean().to_frame()
lot_frontage_dict = neighborhood_average_lf_df.to_dict()
lot_frontage_dict
'''

'\n#Find the dictionary with key as the neighborhood and value as the lot frontage.\nneighborhood_list = []\nfor i in train_dataframe[\'Neighborhood\'].unique():\n    neighborhood_list.append(i)\n\n#create a new dataframe with neighborhoods\ndf = pd.DataFrame(data = neighborhood_list, columns = ["n_name"])\n\n#create a new column for lot frontage\nzero_array = []\nfor i in range(0, len(neighborhood_list)):\n    zero_array.append(0)\n#turn list of zeros into data frame\nzero_df = pd.DataFrame(data = zero_array, columns=["n_avg_frontage"])\n#join neighborhoods and zeros\ndf = pd.concat([df, zero_df], axis = 1)\n\nneighborhood_average_lf_df = train_dataframe.groupby(\'Neighborhood\')[\'LotFrontage\'].mean().to_frame()\nlot_frontage_dict = neighborhood_average_lf_df.to_dict()\nlot_frontage_dict\n'

In [17]:
def neighborhood2(df):
    #replace the neighborhood catagorical variable with the average sale price for that neighborhood. 
    #A way to rank as "good" or "bad" neighborhoods
    neighborhood_dict = {'Neighborhood': {'Blmngtn': 194870.88235294117,
  'Blueste': 137500.0,
  'BrDale': 104493.75,
  'BrkSide': 124834.05172413793,
  'ClearCr': 212565.42857142858,
  'CollgCr': 197965.77333333335,
  'Crawfor': 210624.72549019608,
  'Edwards': 128219.7,
  'Gilbert': 192854.50632911394,
  'IDOTRR': 100123.78378378379,
  'MeadowV': 98576.4705882353,
  'Mitchel': 156270.1224489796,
  'NAmes': 145847.08,
  'NPkVill': 142694.44444444444,
  'NWAmes': 189050.0684931507,
  'NoRidge': 335295.31707317074,
  'NridgHt': 316270.6233766234,
  'OldTown': 128225.30088495575,
  'SWISU': 142591.36,
  'Sawyer': 136793.13513513515,
  'SawyerW': 186555.7966101695,
  'Somerst': 225379.83720930232,
  'StoneBr': 310499.0,
  'Timber': 242247.44736842104,
  'Veenker': 238772.72727272726}}
    df.replace(to_replace=neighborhood_dict, inplace=True)
    print(type(df))
    return df

In [18]:
def lot_frontage_replace(df):
    neighborhood_list = ['CollgCr',
 'Veenker',
 'Crawfor',
 'NoRidge',
 'Mitchel',
 'Somerst',
 'NWAmes',
 'OldTown',
 'BrkSide',
 'Sawyer',
 'NridgHt',
 'NAmes',
 'SawyerW',
 'IDOTRR',
 'MeadowV',
 'Edwards',
 'Timber',
 'Gilbert',
 'StoneBr',
 'ClearCr',
 'NPkVill',
 'Blmngtn',
 'BrDale',
 'SWISU',
 'Blueste']
    
    lot_frontage_dict = {'Neighborhood': {'Blmngtn': 47.142857142857146,
  'Blueste': 24.0,
  'BrDale': 21.5625,
  'BrkSide': 57.509803921568626,
  'ClearCr': 77.5,
  'CollgCr': 70.5625,
  'Crawfor': 70.0952380952381,
  'Edwards': 68.21739130434783,
  'Gilbert': 73.84905660377359,
  'IDOTRR': 60.714285714285715,
  'MeadowV': 27.8,
  'Mitchel': 70.08333333333333,
  'NAmes': 74.85263157894737,
  'NPkVill': 28.25,
  'NWAmes': 77.82978723404256,
  'NoRidge': 91.87878787878788,
  'NridgHt': 81.88157894736842,
  'OldTown': 62.788990825688074,
  'SWISU': 58.91304347826087,
  'Sawyer': 68.71153846153847,
  'SawyerW': 70.09803921568627,
  'Somerst': 64.66666666666667,
  'StoneBr': 62.7,
  'Timber': 80.13333333333334,
  'Veenker': 59.714285714285715}}

    for hood in neighborhood_list:
        for j in range(0, len(df)):
            if df.loc[j, 'Neighborhood'] == hood and np.isnan(df.loc[j, 'LotFrontage']):
                df.loc[j, 'LotFrontage'] = lot_frontage_dict["Neighborhood"][hood]           
    return df

In [19]:
test = lot_frontage_replace(train_dataframe)
test['LotFrontage'].isna().sum()

0


#### Make Ordinal

ExterQual

BsmtQual

BsmntCond

Kitchen Qual

Fireplace Qu

GarageFinish


In [20]:
def ordinal_to_cat_main(df):
    df = ordinal_to_cat(df, 'BsmtQual')
    df = ordinal_to_cat(df, 'BsmtCond')
    df = ordinal_to_cat(df, 'KitchenQual')
    df = ordinal_to_cat(df, 'FireplaceQu')
    df = ordinal_to_cat(df, 'GarageFinish')
    return df

In [21]:
def ordinal_to_cat(df, column_name):
    #extract NA as dummy variable
    cat_exists = []
    for row in df[column_name]:
        if (row is np.nan):
            cat_exists.append(0)
        else:
            cat_exists.append(1)
    new_df = pd.DataFrame(cat_exists, columns=[column_name + '_Exist'])
    new_df = pd.concat([df, new_df], axis=1)
    answer_df = move_SalePrice(new_df)
    
        #replace NA
    return answer_df



In [22]:
#this helper function moves sale price to the end of the list as per convention.
def move_SalePrice(df):
    y = df['SalePrice']
    X = df.loc[:, df.columns != 'SalePrice']
    return pd.concat([X, y], axis=1)

In [23]:
#use this function to find the appropriate value to replace NA with.
def find_na_average(cat):
    td2 = copy.deepcopy(train_dataframe)
    td2.fillna('not_applicable', inplace=True)
    df = td2.groupby([cat]).mean()
    df2 = df['SalePrice'].to_frame()
    average_na =  df2.loc['not_applicable', 'SalePrice']
    median_val = td2['SalePrice'].median()
    answer = 3 * average_na / median_val
    return answer

In [24]:
train_dataframe['BsmtQual'].isnull().sum()

37

In [25]:
find_na_average('GarageFinish')

1.9015451033855941

In [26]:
def cat_to_ordinal(df):
    #replace cat values with ordinal. If NA it fills it in with 3 times the ratio of nas to the median value.
    clean_dict = { 'ExterQual': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4},
                 'BsmtQual': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4, None : find_na_average('BsmtQual')},
                 'BsmtCond': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4, None : find_na_average('BsmtCond')},
                 'KitchenQual': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}, #no NA Values
                 'FireplaceQu': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4, None : find_na_average('FireplaceQu')},
                 'GarageFinish': {None: 0, 'Unf': 1, 'RFn': 2, 'Fin': 3, None : find_na_average('GarageFinish')}
                 }
                 
    df.replace(clean_dict, inplace = True)
        
    return df

In [27]:
result = clean_dataset(train_dataframe)

<class 'pandas.core.frame.DataFrame'>


In [28]:
result.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 98 columns):
Id                    1460 non-null int64
MSSubClass            1460 non-null int64
MSZoning_1            1460 non-null int64
MSZoning_2            1460 non-null int64
MSZoning_3            1460 non-null int64
MSZoning_4            1460 non-null int64
MSZoning_5            1460 non-null int64
LotFrontage           1460 non-null float64
LotArea               1460 non-null int64
Alley_1               1460 non-null int64
Alley_2               1460 non-null int64
Alley_3               1460 non-null int64
LandContour_1         1460 non-null int64
LandContour_2         1460 non-null int64
LandContour_3         1460 non-null int64
LandContour_4         1460 non-null int64
LotConfig_1           1460 non-null int64
LotConfig_2           1460 non-null int64
LotConfig_3           1460 non-null int64
LotConfig_4           1460 non-null int64
LotConfig_5           1460 non-null int64
Neighborh

In [29]:
pd.set_option('display.max_columns', None)

result1 = result.iloc[:, 0:50]
result1.isnull().sum()

Id                0
MSSubClass        0
MSZoning_1        0
MSZoning_2        0
MSZoning_3        0
MSZoning_4        0
MSZoning_5        0
LotFrontage       0
LotArea           0
Alley_1           0
Alley_2           0
Alley_3           0
LandContour_1     0
LandContour_2     0
LandContour_3     0
LandContour_4     0
LotConfig_1       0
LotConfig_2       0
LotConfig_3       0
LotConfig_4       0
LotConfig_5       0
Neighborhood_0    0
Neighborhood_1    0
Neighborhood_2    0
Neighborhood_3    0
Neighborhood_4    0
Neighborhood_5    0
Condition1        0
Condition2        0
BldgType          0
HouseStyle        0
OverallQual       0
OverallCond       0
YearBuilt         0
YearRemodAdd      0
RoofMatl          0
Exterior1st       0
MasVnrType_1      0
MasVnrType_2      0
MasVnrType_3      0
MasVnrType_4      0
MasVnrType_5      0
MasVnrArea        8
ExterQual         0
ExterCond         0
Foundation_1      0
Foundation_2      0
Foundation_3      0
Foundation_4      0
Foundation_5      0


In [30]:
result2 = result.iloc[:,50:120]
result2.isnull().sum()

Foundation_6           0
BsmtQual               0
BsmtCond               0
BsmtFinType1          37
BsmtFinSF1             0
BsmtFinSF2             0
BsmtUnfSF              0
TotalBsmtSF            0
HeatingQC              0
CentralAir             0
Electrical             1
FirstFlrSF             0
SecondFlrSF            0
LowQualFinSF           0
GrLivArea              0
BsmtFullBath           0
BsmtHalfBath           0
FullBath               0
HalfBath               0
BedroomAbvGr           0
KitchenAbvGr           0
KitchenQual            0
TotRmsAbvGrd           0
Fireplaces             0
FireplaceQu            0
GarageType            81
GarageYrBlt           81
GarageFinish           0
GarageCars             0
GarageArea             0
PavedDrive             0
WoodDeckSF             0
OpenPorchSF            0
EnclosedPorch          0
ScreenPorch            0
PoolArea               0
MiscVal                0
MoSold                 0
YrSold                 0
SaleType               0
